# Puppeteer 사용법 및 Chrome for Developers 웹사이트 크롤링하는 방법 알아보기
---
크롤링(Crawling)을 할 수 있는 도구라고 하면 대개 셀레니움(Selenium)을 떠올릴테지만, Node.js 기반의 퍼페티어(Puppeteer)라는 라이브러리를 사용할 수도 있습니다. 전자의 경우에는 C#, Rupy, JAVA, Python, JavaScript 언어를 지원하지만, 후자의 경우에는 자바스크립트만을 지원하는 라이브러리라는 차이점이 있지요. 그 외에도 소소한 차이점이 있지만, 대개 자바스크립트 개발자, 그중에서 크롬(Chrome) 웹브라우저를 주로 사용하는 사람이면 셀레니움보다는 퍼페티어를 사용하는 것을 선호하는 모양입니다.
</p></br></br>

이번에는 자바스크립트를 이용해서 퍼페티어를 설치하고, [Chrome for Developers](https://developer.chrome.com/?hl=ko) 웹사이트를 크롤링해보는 작업을 해볼 예정입니다. 이 글은 [퍼페티어 공식 홈페이지](https://pptr.dev/)를 번역 및 재구성하였습니다.
</p></br></br>

## Puppeteer 설치 및 불러오기
---
만약 내 컴퓨터에 퍼페티어가 설치되지 않았다면, npm 패키지 관리자를 이용해서 설치해 주면 됩니다. 설치를 위한 명령어는 ```npm i puppeteer``` 입니다.
</p></br></br>

## Puppeteer에서 지원하는 메소드
---
크롤링을 위해서는 브라우저 조작에 대한 메소드 몇 가지를 외워둬야 합니다. 창을 켜고 끄는 작업, 그리고 특정 URL로 이동하거나 대상을 선택하는 작업이지요. 퍼페티어에서 제공하는 주요 메소드를 외워두고 상황에 맞게 사용해 보시기 바랍니다.
</p></br></br>

* `puppeteer.launch()`: 브라우저 초기화
* `BrowserContext.newPage()`: 새 창 열기
* `Page.goto(URL)`: 입력된 URL로 이동하기
* `Page.type(field, text)`: field 객체에 text 문자를 입력
* `Page.waitForSelector(target)`: target의 로드가 완료될 때까지 명시적 대기
* `Page.click(target)`: target 객체를 클릭
</p></br></br>

## Chrome for Developers 크롤링하기
---
이제 퍼페티어를 이용해서 Chrome for Developers 웹사이트를 크롤링해 보도록 하겠습니다. 해당 사이트로 접속한 뒤, **automate Beyond Recorder** 라는 검색어로 검색한 결과 중 가장 위에 있는 글의 제목을 저장하는 방식입니다. 해당 기능을 구현하기 위해서는, 접속할 웹사이트 주소 및 Chrome for Developers에서 검색창의 클래스 이름을 찾아둔 다음 검색어를 입력해야겠지요. 해당 코드는 각각 아래와 같이 구현할 수 있습니다. 참고로, 검색창의 클래스 이름을 찾기 위해서는 크롬 개발자 도구를 이용하여 input 태그에 할당되어 있는 클래스 이름을 찾아보셔야 합니다.
</p></br></br>

```javascript
page.goto('https://developer.chrome.com/?hl=ko');
page.type('.devsite-search-field', 'automate beyond recorder');
```
</p></br></br>

다음으로는, 검색 결과를 대기한 다음 해당 결과 중 `Customize and automate` 라는 글을 찾아서 제목을 로깅하는 코드를 구현합니다. 전체 코드는 아래와 같이 구성할 수 있어요.
</p></br></br>



In [1]:
// Import library
const puppeteer = require('puppeteer');

(async () => {
  // Launch the browser and open a new blank page
  const browser = await puppeteer.launch();
  const page = await browser.newPage();

  // Navigate the page to a URL
  await page.goto('https://developer.chrome.com/?hl=ko');

  // Set screen size
  await page.setViewport({width: 1080, height: 1024});

  // Type into search box
  await page.type('.devsite-search-field', 'automate beyond recorder');

  // Wait and click on first result
  const searchResultSelector = '.devsite-result-item-link';
  await page.waitForSelector(searchResultSelector);
  await page.click(searchResultSelector);

  // Locate the full title with a unique string
  const textSelector = await page.waitForSelector(
    'text/Customize and automate'
  );
  const fullTitle = await textSelector?.evaluate(el => el.textContent);

  // Print the full title
  console.log('The title of this blog post is "%s".', fullTitle);

  await browser.close();
})();

Promise { <pending> }

The title of this blog post is "Customize and automate user flows beyond Chrome DevTools ...".


</p></br></br>

## 브라우저 창 조절
---
크롤러의 경우, 사용자의 개입 없이 자동으로 웹사이트 크롤링을 수행하기 때문에 헤드리스 모드를 사용하는 경우가 많습니다. 헤드리스 모드란, 웹사이트 접속을 할 때 별도의 창을 띄우지 않고 작업하는 것이지요. 퍼페티어의 경우, 헤드리스 모드가 기본값으로 설정되어 있어서 크롤러 프로그램을 테스트하는 환경에서는 눈으로 동작을 확인할 수 없습니다. 만약 테스트를 위해서 헤드리스 모드를 해제하고 싶은 경우, 브라우저를 초기화할 때 `headless: false` 옵션을 부여하는 방식으로 해결할 수 있습니다. 해당 코드는 아래와 같이 작성할 수 있어요.
</p></br></br>

```javascript
puppeteer.launch({headless: false})
```
</p></br></br>

그리고, 창 크기에 따라서 보이는 객체가 달라지는 반응형 웹사이트의 경우, 고정된 창 크기로 크롤링을 하는 것이 유리합니다. 이 때는, 창 크기를 조절할 수 있는 메소드를 이용해 봐야겠지요.
</p></br></br>

```javascript
Page.setViewport({width: w, height: h})
````
</p></br></br>

해당 메소드는 가로 w 픽셀, 세로 h 픽셀 크기로 창 크기를 조절할 수 있도록 해 줍니다. 대부분의 PC 웹은 1280\*720px 크기 이상인 경우 레이아웃 문제가 발생하지 않으며, 웹사이트에 따라 1920\*1080px 또는 2560\*1440px 정도의 크기가 나와야 모든 객체가 노출되는 구조로 설계된 경우도 있습니다. 해당 크기보다 작은 경우, 특히 가로 폭이 짧은 경우에는 웹사이트 설계에 따라 태블릿pc용 레이아웃 또는 모바일용 레이아웃으로 변경될 수 있습니다.
</p></br></br>

마지막으로, 크롤링이 끝난 뒤 열려 있는 크롬 창을 닫고 싶은 경우에는 `BrowserContext.close()` 메소드를 이용해 창을 닫을 수 있습니다. 만약 제때 창을 닫지 않는 경우, 계속해서 크롬 창이 실행된 상태로 유지되기 때문에 메모리 관리 및 예상치 못한 문제가 생길 수 있습니다.